In [1]:
import numpy as np
from typing import Callable, List
import matplotlib
import matplotlib.pyplot as plt
from tinygrad.tensor import Tensor
from tinygrad import nn
from tinygrad.nn.optim import Optimizer

# from tinygrad.extra.lr_scheduler import LR_Scheduler, ReduceLROnPlateau

In [2]:
# CONFIG

DEGREE = 5 # degree of polynomial to fit

# Plan

Let's hack the insolubility of computing the roots of a polynomial of degree 5,
 by using neural networks

In [3]:
def plot(x: np.ndarray) -> Callable[[np.ndarray,str,str], None]:
    """
    Curries X into the plot function
    """
    def fn(y: np.ndarray, label: str, color: str) -> None:
        plt.plot(x, y, label=label, color=color)
    return lambda y,label,color : fn(y,label,color)


def plt_setup(xlim:tuple = (0,1), ylim:tuple = (0,1), title:str = "getting $a*(X^2)+b$ from Relus is the goal") -> None:
    plt.grid(True)
    # plt.style.use('dark_background')
    plt.axhline(y=0, color='k')
    plt.axvline(x=0, color='k')
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.title(title)
    plt.style.use('dark_background')
    return None

color_cycle = matplotlib.colormaps["Spectral"]

X=np.arange(0,1,0.001)
p = plot(X)

# from tinygrad.extra
class LR_Scheduler:
  def __init__(self, optimizer: Optimizer):
    self.optimizer = optimizer
    self.epoch_counter = Tensor([0], requires_grad=False, device=self.optimizer.device)

  def get_lr(self): pass

  def step(self) -> None:
    self.epoch_counter.assign(self.epoch_counter + 1).realize()
    self.optimizer.lr.assign(self.get_lr()).realize()

class OneCycleLR(LR_Scheduler):
  def __init__(self, optimizer: Optimizer, max_lr: float, div_factor: float, final_div_factor: float, total_steps: int, pct_start: float):
    super().__init__(optimizer)
    self.initial_lr = max_lr / div_factor
    self.max_lr = max_lr
    self.min_lr = self.initial_lr / final_div_factor
    self.total_steps = total_steps
    self.pct_start = pct_start
    self.optimizer.lr.assign(self.get_lr()).realize() # update the initial LR

  @staticmethod
  def _annealing_linear(start: float, end: float, pct: Tensor) -> Tensor: return (pct*(end-start)+start)

  def get_lr(self) -> Tensor:
    return (self.epoch_counter < self.total_steps*self.pct_start).where(
      self._annealing_linear(self.initial_lr, self.max_lr, self.epoch_counter/(self.total_steps*self.pct_start)),
      self._annealing_linear(self.max_lr, self.min_lr, (self.epoch_counter-(self.total_steps*self.pct_start))/(self.total_steps*(1-self.pct_start)))
    )
    
class Model:
  # inputs : a, b, c, d, e (coefficients of polynomial)  -> actually just b,c,d,e because a is always 1
  # expected output = roots z1, z2, z3, z4, z5 (assumed to be reals for now)

  def __init__(self, layers:int = 3):
    self.layers = [nn.Linear(5, 3), Tensor.relu] + [nn.Linear(3, 3), Tensor.relu]*layers + [nn.Linear(3, DEGREE), Tensor.sigmoid]

  def __call__(self, x:Tensor) -> Tensor: return x.sequential(self.layers)

  def L1(self) -> Tensor: return sum([l.weight.abs().sum() + l.bias.abs().sum() for l in self.layers if isinstance(l, nn.Linear)])

  def L2(self) -> Tensor: return sum([l.weight.square().sum() + l.bias.square().sum() for l in self.layers if isinstance(l, nn.Linear)])


def train_step(x:Tensor, y:Tensor, model: Model, opt: nn.optim.LAMB, lr_schedule: LR_Scheduler) -> Tensor:
    y_pred = model(x)
    loss = (y_pred - y).square().mean() #+ 0.0001 * (model.L2())# + 0.0001 * (model.L2())
    opt.zero_grad()
    loss.backward()
    opt.step()
    lr_schedule.step()
    return loss


def plot_model(model, title:str = "neural network learning versus the takagi curve") -> None:
    #p(target(Tensor(X.astype(np.float32)).reshape(-1,1)).numpy(), "target", color=color_cycle(0.0))
    p(model(Tensor(X.astype(np.float32)).reshape(-1,1)).numpy(), "model", color=color_cycle(0.5))
    #p(T_1(X)+T_2(X)+T_3(X), "Takagi 3", color=color_cycle(0.75))
    plt_setup(xlim=(0,1), ylim=(0,1/2+0.1), title=title)
    plt.legend(loc='best')
    plt.show()



In [4]:
from itertools import combinations
roots = [1,2,3,4,5]
print(sum(r1 * r2 for r1, r2 in combinations(roots, 2)))


85


In [5]:
Tensor.ones((1,5)).numpy()

for i in range(5):
    print(Tensor.ones((1,5))[:,i].numpy())

[1.]
[1.]
[1.]
[1.]
[1.]


In [6]:
help(Tensor.stack)

Help on function stack in module tinygrad.tensor:

stack(tensors: 'Sequence[Tensor]', dim: 'int' = 0) -> 'Tensor'



In [8]:
# we write the quintic polynomial P(z)
# (z-z1)(z-z2)(z-z3)(z-z4)(z-z5)
""" 
Given a polynomial of degree 5: Vieta's formulas.
$p(x) = x^5 + a_4x^4 + a_3x^3 + a_2x^2 + a_1x + a_0$

where the roots are r_1, r_2, r_3, r_4, r_5, the coefficients can be calculated as follows:

- $a_4 = -(r_1 + r_2 + r_3 + r_4 + r_5)$
- $a_3 = r_1r_2 + r_1r_3 + r_1r_4 + r_1r_5 + r_2r_3 + r_2r_4 + r_2r_5 + r_3r_4 + r_3r_5 + r_4r_5$
- $a_2 = -(r_1r_2r_3 + r_1r_2r_4 + r_1r_2r_5 + r_1r_3r_4 + r_1r_3r_5 + r_1r_4r_5 + r_2r_3r_4 + r_2r_3r_5 + r_2r_4r_5 + r_3r_4r_5)$
- $a_1 = r_1r_2r_3r_4 + r_1r_2r_3r_5 + r_1r_2r_4r_5 + r_1r_3r_4r_5 + r_2r_3r_4r_5$
- $a_0 = -r_1r_2r_3r_4r_5$

# Example usage
coefficients = polynomial_coefficients(1, 2, 3, 4, 5)
print(coefficients)
"""
from itertools import combinations


#function decaorator that prints arguments and function name
def print_args(func):
    def inner(*args, **kwargs):
        print(f"called {func.__name__}\n"
              f"with args: {args}, kwargs: {kwargs}")
        return_value = func(*args, **kwargs)
        print(f"{func.__name__} returns {return_value}")
        return return_value
    return inner


# @print_args
def polynomial_coefficients(roots: Tensor) -> Tensor:
    roots = [roots[:,i] for i in range(5)]
    if len(roots) != 5:
        raise ValueError(f"Exactly 5 roots are required for a polynomial of degree 5., got {len(roots)}, got roots = {roots}")
    a4 = -sum(roots)
    a3 = sum(r1*2 for r1, r2 in combinations(roots, 2))
    a2 = -sum(r1*r2*r3 for r1, r2, r3 in combinations(roots, 3))
    a1 = sum(r1*r2*r3*r4 for r1, r2, r3, r4 in combinations(roots, 4))
    a0 = -roots[0]*roots[1]*roots[2]*roots[3]*roots[4]
    return Tensor.stack([a4, a3, a2, a1, a0],dim=1)


# step 1 train a neural net to learn the roots
def train_model(model, lr:float = 0.01, steps:int = 1001, bs:int = 32768) -> Model:
    opt = nn.optim.Adam(nn.state.get_parameters(model), lr)
    lr_schedule = OneCycleLR(opt, max_lr=0.1, div_factor=100, final_div_factor=100, total_steps=steps, pct_start=0.5)
    old_lr = opt.lr.numpy()
    for i in range(steps):
        roots = (20.0*Tensor.rand(bs, 5)-10.0).realize() # sample the roots between -10 and 10
        decoded_inputs = polynomial_coefficients(roots) # see if we can call this on a Tensor ? 
        loss = train_step(x=decoded_inputs, y=roots, model=model, opt=opt, lr_schedule=lr_schedule)
        if i%100 == 0:
            print(f"lr = {opt.lr.numpy()[0]}")
            print(f"loss at train_step = {i} : {loss.numpy()}")
    return model

# step 3 apply it to a quintic polynomial

model = train_model(Model(layers=25))
  
# step 4 use newton's method to find the roots given the initialization point given by the neural network

lr = 0.0011978022521361709
loss at train_step = 0 : 33.457515716552734
lr = 0.020978020504117012
loss at train_step = 100 : 33.3631706237793
lr = 0.04075824096798897
loss at train_step = 200 : 33.33977508544922
lr = 0.060538459569215775
loss at train_step = 300 : 33.20367431640625
lr = 0.08031868189573288
loss at train_step = 400 : 33.43901443481445
lr = 0.09990011155605316
loss at train_step = 500 : 33.30876922607422
lr = 0.07992208749055862
loss at train_step = 600 : 33.2637825012207
lr = 0.059944067150354385
loss at train_step = 700 : 33.391746520996094
lr = 0.039966046810150146
loss at train_step = 800 : 33.44405746459961
lr = 0.01998802460730076
loss at train_step = 900 : 33.3244514465332
lr = 1.0002695489674807e-05
loss at train_step = 1000 : 33.444541931152344
